<a href="https://colab.research.google.com/github/dahlia52/Advanced-Statistical-Data-Analysis/blob/main/softmax_classification_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()]) # 이미지를 텐서로 변환

# Prepare Data
train_data = CIFAR100(root = path, train = True, transform = transform, download = True)
test_data = CIFAR100(root = path, train = False, transform = transform, download = True)

batch_size = 100

# DataLoader
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True, num_workers = 4)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False, num_workers = 4)

print(train_data)
print(test_data)

input_shape = train_data[0][0].reshape(-1).shape[0] # 3072 = 3*32*32
output_shape = len(train_data.classes) # 100

print(input_shape,output_shape)

100%|██████████| 169001437/169001437 [00:01<00:00, 87788813.86it/s]


Extracting ./datasets/cifar-100-python.tar.gz to ./datasets/
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./datasets/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./datasets/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )
3072 100


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps:0")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu:0")

print(device)

cuda:0


In [4]:
from torch.nn.modules.activation import LeakyReLU
class SoftmaxClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_shape, 1728),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(1728, 512),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(128, output_shape)
    )

  def forward(self, x):
    return self.layers(x)

In [5]:
model = SoftmaxClassifier().to(device)
loss = nn.CrossEntropyLoss(reduction = 'sum')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-3)

In [6]:
num_epoch = 50
train_loss_list, test_loss_list = list(), list()

for i in range(num_epoch):
  # train
  model.train()

  total_loss = 0
  count = 0

  for batch_idx, (x, y) in enumerate(train_loader):
    x, y = x.to(device), y.to(device)
    y_est = model.forward(x)
    cost = loss(y_est, y)

    total_loss += cost.item()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    pred = torch.argmax(y_est, dim = -1)
    count += (pred == y).sum().item()

  acc = count / len(train_data)
  avg_loss = total_loss / len(train_data)

  train_loss_list.append(avg_loss)

  if i % 1 == 0:
        print("\nEpoch %d Train: Loss %.3f / Accuracy %.3f"%(i,avg_loss,acc))

  # test
  model.eval()

  total_loss = 0
  count = 0

  with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
      x, y = x.to(device), y.to(device)
      y_est = model.forward(x)
      cost = loss(y_est, y)

      total_loss += cost.item()

      pred = torch.argmax(y_est, dim = -1)
      count += (pred == y).sum().item()

    acc = count / len(test_data)
    avg_loss = total_loss / len(test_data)

    test_loss_list.append(avg_loss)

    if i % 1 == 0:
      print("Epoch %d Test: Loss %.3f / Accuracy %.3f"%(i,avg_loss,acc))


Epoch 0 Train: Loss 4.609 / Accuracy 0.011
Epoch 0 Test: Loss 4.581 / Accuracy 0.014

Epoch 1 Train: Loss 4.450 / Accuracy 0.023
Epoch 1 Test: Loss 4.296 / Accuracy 0.047

Epoch 2 Train: Loss 4.303 / Accuracy 0.036
Epoch 2 Test: Loss 4.167 / Accuracy 0.055

Epoch 3 Train: Loss 4.234 / Accuracy 0.044
Epoch 3 Test: Loss 4.136 / Accuracy 0.063

Epoch 4 Train: Loss 4.186 / Accuracy 0.050
Epoch 4 Test: Loss 4.077 / Accuracy 0.067

Epoch 5 Train: Loss 4.166 / Accuracy 0.053
Epoch 5 Test: Loss 4.042 / Accuracy 0.076

Epoch 6 Train: Loss 4.143 / Accuracy 0.056
Epoch 6 Test: Loss 4.024 / Accuracy 0.076

Epoch 7 Train: Loss 4.124 / Accuracy 0.057
Epoch 7 Test: Loss 4.013 / Accuracy 0.079

Epoch 8 Train: Loss 4.111 / Accuracy 0.062
Epoch 8 Test: Loss 3.988 / Accuracy 0.091

Epoch 9 Train: Loss 4.098 / Accuracy 0.063
Epoch 9 Test: Loss 3.992 / Accuracy 0.087

Epoch 10 Train: Loss 4.084 / Accuracy 0.066
Epoch 10 Test: Loss 3.958 / Accuracy 0.094

Epoch 11 Train: Loss 4.080 / Accuracy 0.066
Epoch 1

In [7]:
num_parameter = 0

for parameter in model.parameters():
  print(parameter.shape)
  num_parameter += np.prod(parameter.size())
print(num_parameter)

torch.Size([1728, 3072])
torch.Size([1728])
torch.Size([512, 1728])
torch.Size([512])
torch.Size([256, 512])
torch.Size([256])
torch.Size([128, 256])
torch.Size([128])
torch.Size([100, 128])
torch.Size([100])
6372516


- 파라미터 개수는 더 많지만, LeNet보다 성능이 안좋음.